In [4]:
// Import the API, Keyring and some utility functions
const { ApiPromise } = require('@polkadot/api');
const { Keyring } = require('@polkadot/keyring');

const BOB = '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty';

// Instantiate the API
const api = await ApiPromise.create();

// Constuct the keying after the API (crypto has an async init)
const keyring = new Keyring({ type: 'sr25519' });

// Add Alice to our keyring with a hard-deived path (empty phrase, so uses dev)
const alice = keyring.addFromUri('//Alice');
alice

Unknown types found, no types for Releases


{ address: [Getter],
  isLocked: [Getter],
  meta: [Getter],
  publicKey: [Getter],
  type: [Getter],
  decodePkcs8: [Function: decodePkcs8],
  derive: [Function: derive],
  encodePkcs8: [Function: encodePkcs8],
  lock: [Function: lock],
  setMeta: [Function: setMeta],
  sign: [Function: sign],
  toJson: [Function: toJson],
  verify: [Function: verify] }


In [5]:
// The actual address that we will use
const ADDR = '5DTestUPts3kjeXSTMyerHihn1uwMfLj8vU8sqF7qYrFabHE';

// Retrieve the last timestamp
const now = await api.query.timestamp.now();

// Retrieve the account balance & nonce via the system module
const { nonce, data: balance } = await api.query.system.account(ADDR);

console.log(`${now}: balance of ${balance.free} and a nonce of ${nonce}`);

1598647662000: balance of 0 and a nonce of 0


In [6]:
// Retrieve last block timestamp, account nonce & balances
const [now, { nonce, data: balances }] = await Promise.all([
  api.query.timestamp.now(),
  api.query.system.account(ADDR)
]);

now

<BN: 17436d605f4>


### RPC查询
RPC调用为往来于节点的数据传输提供了主干。这意味着所有API端点，例如api.query，api.tx或者api.derive只是包装RPC调用，由节点如预期提供的编码格式信息。

In [7]:
// Retrieve the chain name
const chain = await api.rpc.system.chain();

// Retrieve the latest header
const lastHeader = await api.rpc.chain.getHeader();

// Log the information
console.log(`${chain}: last block #${lastHeader.number} has hash ${lastHeader.hash}`);

Development: last block #3044 has hash 0x429d7f06ba41403eb03238a9a019e3e894785a2eab58a7639a3a1e37aa9b17a0


In [8]:
// 无论何时创建订阅，我们都希望自己清理并取消订阅，因此，假设我们只想记录前10个标头
let count = 0;

// Subscribe to the new headers
const unsubHeads = await api.rpc.chain.subscribeNewHeads((lastHeader) => {
  console.log(`${chain}: last block #${lastHeader.number} has hash ${lastHeader.hash}`);

  if (++count === 10) {
    unsubHeads();
  }
});

Development: last block #3064 has hash 0xc0e58044e5bb6cedcdeeabbddf88aa0efd306adea7cf71003e146abc7d4b596e
Development: last block #3065 has hash 0x19df6b62216766eab9c7570f4371cca82ed617362d729236c13a0b85cf13bc2c
Development: last block #3066 has hash 0xc9981b0e16ba1f32b7ae7af6442d30cbb9e2ca4b06bab7175e5c4c6f536fb87c
Development: last block #3067 has hash 0x5ba725df92dbbfb5b3e95a4e9c6b8c39c47aeee77e43ed098abde5a73b66851d
Development: last block #3068 has hash 0x0f7b4713a1d56bfc9171a0b9e4d59016f36013cfeccefb00365da8a1cf5af382
Development: last block #3069 has hash 0xa9e43d2c5244316c87b80de6c32660033c2591e0b5fec8f9eab0f6c695167c84
Development: last block #3070 has hash 0xe3bdac110f8ce7e5f4b883ab121e71254724ab669576702007aad76089a92bc0
Development: last block #3071 has hash 0x2665b1de808b4b7c5b82795fd4c49b08ffcc2e8026f2461fc83442ae049ef1b8
Development: last block #3072 has hash 0x02f4a69599de51e7be0fec832f1329100ccc52617c236d22b432f153fe6524a8
Development: last block #3073 has hash 0x19a99

In [9]:
const unsub = await api.derive.chain.subscribeNewHeads((lastHeader) => {
  console.log(`#${lastHeader.number} was authored by ${lastHeader.author}`);
});

#3086 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3087 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3088 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3089 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3090 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3091 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3092 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3093 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3094 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3095 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3096 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3097 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3098 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3099 was authored by 5GNJqTPyNqANBkUVMN1LPPrxXnFouWXoe2wNSmmEoLctxiZY
#3100 

In [ ]:
unsub()